In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import plot_tree

from sklearn.model_selection import train_test_split
# Regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
# Clasification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, accuracy_score
# Best_searches
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
# Standarization
from sklearn.preprocessing import StandardScaler

from keras import models
from keras.layers import Dense

In [2]:
df_train = pd.read_csv('data/train.csv', encoding='utf8')
df_test = pd.read_csv('data/test.csv', encoding='utf8')
df_final = pd.read_csv('data/sample_submission.csv', encoding='utf8')

In [55]:
df_train.describe()

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
mean,68214.000000,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,136428.000000,304.400000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [52]:
df_test[df_test['PWF'] != 0]

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
405,136834,L47380,L,298.4,308.6,2636,12.8,103,0,0,1,0,0
576,137005,L56355,L,297.9,309.0,1403,64.2,0,0,0,1,0,0
950,137379,L49935,L,299.7,309.3,1478,38.8,8,0,0,1,0,0
1570,137999,L55609,L,299.0,310.4,2861,4.6,92,0,0,1,0,0
2653,139082,L53074,L,301.2,311.0,1267,69.3,164,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86883,223312,M20301,M,303.0,312.6,1363,62.9,119,0,0,1,0,0
87045,223474,L49512,L,299.2,308.5,2232,15.9,220,0,0,1,0,0
89247,225676,M18633,M,302.4,311.1,2659,10.8,139,0,0,1,0,0
90239,226668,L51961,L,303.4,311.7,1374,63.7,121,0,0,1,0,0


In [4]:
df_train.shape

(136429, 14)

In [54]:
df_train[df_train['Machine failure'] != 0].shape

(2148, 14)

## Opis danych: 

id - 
    Wiemy o co chodzi
    
Podcut ID - 
    Pomysły: Zmapować konkretny produkt id, jako taki z dużą lub małą szansą na uszkodzenie. 
    
Type - ( H, M, L )(High, Medium, Low) - 
    Im wyższy tym mniejsza szansa na uszkodzenie i większa tolerancja na długotrwałą pracę
    
Air temparture - temperatura powietrza
    zakres: 295 - 304 stopnie, median 300
    Różnica między tą temperaturą a temperaturą procesu może wskazywać na przegrzewanie.
    Pomysły: Zakodować różnicę między temperaturami.
    
Process temperature - temperatura pracy
    zakres 305 - 314, median 310.
    Pomysły: Sprawdzić zależność między awariami a tą temperaturą, DLA WSZYSTKICH TYPU AWARII
    
<i>Jeżeli Air Temprature będzie równe Process temp lub większe, użądzenie nie będzie się w stanie schłodzić i się nagrzeje.</i> 

Rotational speed rpm - prędkość rotacji/obrotu
    zakres 1181 - 2886, 996 unikalnych
    Pomysł: Porównać z Tool Wear 
    
Torque Nm - Moment obrotowy 
    Pomysł: użyć wzór, spróbować oszacować błąd. Power (kW) = Torque (N.m) x Speed (RPM) z TWF
    Możliwe: powiązanie OSF
    
Tool wear - Czas sesji użycia narzędzia
    Zakres 0 - 253
    Pomysł: Im większy tool wear, może powodować awarie
    
Typy awarii:
    TWF - Zużycie
    HDF - Przegrzanie
    PWF - Za mała moc
    OSF - Przeciążenie mechaniczne
    RNF - Losowe
    Pomysły: Zobaczyć zależności pomiędzy tymi typami awarii i dokładnie je zbadać.

In [46]:
df_train.describe()

,id,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
count,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000,136429.000000
mean,68214.000000,299.862776,309.941070,1520.331110,40.348643,104.408901,0.015744,0.001554,0.005160,0.002397,0.003958,0.002258
std,39383.804275,1.862247,1.385173,138.736632,8.502229,63.965040,0.124486,0.039389,0.071649,0.048899,0.062789,0.047461
min,0.000000,295.300000,305.800000,1181.000000,3.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,34107.000000,298.300000,308.700000,1432.000000,34.600000,48.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,68214.000000,300.000000,310.000000,1493.000000,40.400000,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,102321.000000,301.200000,310.900000,1580.000000,46.100000,159.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,136428.000000,304.400000,313.800000,2886.000000,76.600000,253.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [30]:
df_train['Product ID'].unique().shape

(9976,)

In [11]:
df_train['Type'].unique()

array(['L', 'M', 'H'], dtype=object)

In [38]:
df_train['Air temperature [K]']

299.8627757661494

In [17]:
df_train['Process temperature [K]'].unique()

array([309.6, 312.1, 308.5, 310.9, 309. , 308.9, 311. , 310.1, 308.8,
       312.3, 310.8, 310.7, 308.4, 309.4, 308.3, 309.9, 309.8, 311.7,
       312.4, 310.4, 307.9, 312.2, 308.2, 309.5, 309.3, 308.1, 311.9,
       308. , 310.6, 309.1, 307.5, 312. , 308.6, 310.2, 311.2, 311.6,
       310.5, 309.7, 311.1, 311.4, 313.1, 312.7, 308.7, 311.3, 311.8,
       310.3, 310. , 311.5, 312.8, 313.2, 307.7, 307.8, 306. , 313. ,
       313.4, 309.2, 306.8, 306.1, 307.2, 312.5, 312.9, 306.4, 306.9,
       307.4, 307.6, 307.1, 306.5, 313.3, 306.2, 307.3, 312.6, 313.5,
       306.6, 313.6, 306.3, 305.8, 305.9, 307. , 306.7, 313.8, 313.7])

In [49]:
sorted(df_train['Tool wear [min]'].unique())

[0,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [19]:
df_train['Rotational speed [rpm]'].unique().shape

(952,)

In [22]:
df_train['Torque [Nm]'].unique().shape

(611,)

In [26]:
df_train['Tool wear [min]'].unique()

array([140, 200,  25, 197,  34,  65, 156,  67,  13,  98, 139, 166, 152,
       204, 143,  38,  72,   5,   0,  76,  40,  27,  23, 190, 173, 162,
         2, 124,  51, 111, 129,  71,  89, 158, 118,  19, 119,  94,  83,
       172, 103, 135,  14, 155, 161,   9,  20, 183, 201,  90, 144, 206,
       131, 168,  53,  99,  70, 211,  93,  37,  64, 145, 106,  80,  77,
       127, 141,  47, 154, 196, 100,  32,  96,  35, 146,  30, 130, 110,
       120, 192,  59, 147, 101,  46,  10,  91, 138, 176, 104,  52,  88,
       150,  44,  18,  16,  36,  21,  26,  24,  56, 128, 148, 164, 149,
       209, 219,  15, 174, 132,  68,  79, 215,  17, 182,  81,  50,   3,
       137, 102,  45,  43,  75, 136,  33, 188, 167, 122,  22, 186,  54,
        62, 116, 189, 157, 222, 159, 178,  55, 210,  92,   6,  73, 160,
       242,  48,  12, 123, 175, 187, 198, 134,  28,  66,  41, 142,  97,
       185,  63, 199,  49,  74, 179, 195, 205, 203,  84, 113, 181, 228,
        39,  42, 126, 125,  60, 108,   7,  82, 208, 193,  61,  5